# Pandas (2)

In [1]:
import pandas as pd
import numpy as np

## Лабораторная работа №2.2

In [12]:
recipes = pd.read_csv('recipes_sample.csv', sep = ',')
reviews = pd.read_csv('reviews_sample.csv', sep = ',', index_col = 0)

### Применение функций к pd.Series и pd.DataFrame

4.1 Напишите функцию, которая переводит минуты в формат "XhYm". Примените эту функцию к столбцу `minutes` таблицы `recipes` (без перезаписи исходного столбца) при помощи метода `map`.

In [13]:
def m_to_hm(m: int) -> str:
    return(str((m % 1440)// 60) + 'h' + str((m % 1440) % 60) + 'm')

assert m_to_hm(90) == "1h30m"
assert m_to_hm(10) == "0h10m"

In [14]:
recipes['minutes'].map(m_to_hm)

0        1h30m
1        0h10m
2        0h30m
3        0h45m
4        0h25m
         ...  
29995    1h20m
29996     4h0m
29997    1h15m
29998     1h0m
29999    0h29m
Name: minutes, Length: 30000, dtype: object

4.2 На основе таблицы `recipes` создайте таблицу, которая содержит только текстовые столбцы (используйте метод `select_dtypes`).  Примените к каждому элементу этой таблицы строковый метод `str.capitalize` при помощи метода `applymap`, не удаляя пропуски.

In [15]:
recipes_object = recipes.select_dtypes(include=['object'])
recipes_object

,name,submitted,description
0,george s at the cove black bean soup,2002-10-25,an original recipe created by chef scott meska...
1,healthy for them yogurt popsicles,2003-07-26,my children and their friends ask for my homem...
2,i can t believe it s spinach,2002-08-29,"these were so go, it surprised even me."
3,italian gut busters,2002-07-27,my sister-in-law made these for us at a family...
4,love is in the air beef fondue sauces,2004-02-23,i think a fondue is a very romantic casual din...
...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,2007-11-25,this is based on a french recipe but i changed...
29996,zwetschgenkuchen bavarian plum cake,2009-08-24,"this is a traditional fresh plum cake, thought..."
29997,zwiebelkuchen southwest german onion cake,2004-11-03,this is a traditional late summer early fall s...
29998,zydeco soup,2012-08-29,this is a delicious soup that i originally fou...


In [16]:
recipes_object.applymap(lambda x: x.capitalize(), na_action='ignore')

,name,submitted,description
0,George s at the cove black bean soup,2002-10-25,An original recipe created by chef scott meska...
1,Healthy for them yogurt popsicles,2003-07-26,My children and their friends ask for my homem...
2,I can t believe it s spinach,2002-08-29,"These were so go, it surprised even me."
3,Italian gut busters,2002-07-27,My sister-in-law made these for us at a family...
4,Love is in the air beef fondue sauces,2004-02-23,I think a fondue is a very romantic casual din...
...,...,...,...
29995,Zurie s holey rustic olive and cheddar bread,2007-11-25,This is based on a french recipe but i changed...
29996,Zwetschgenkuchen bavarian plum cake,2009-08-24,"This is a traditional fresh plum cake, thought..."
29997,Zwiebelkuchen southwest german onion cake,2004-11-03,This is a traditional late summer early fall s...
29998,Zydeco soup,2012-08-29,This is a delicious soup that i originally fou...


4.3 Напишите функцию, которая принимает на вход серию `pd.Series` и для серий, содержащих текстовые данные, возвращает максимальную длину строк в ней, а для числовых серий возвращает минимальный элемент в этой серии. Примените данную функцию к каждому столбцу таблицы `recipes` при помощи метода `apply`.

In [17]:
def get_stats(x: pd.Series) -> int:
    
    stats_object = pd.DataFrame(x).select_dtypes(include=['object'])
    stats_number = pd.DataFrame(x).select_dtypes(include=['number'])
    
    if pd.DataFrame(stats_object).select_dtypes(include=['object']).empty and\
        pd.DataFrame(stats_number).select_dtypes(include=['number']).empty:
        return np.NaN
    if pd.DataFrame(stats_object).select_dtypes(include=['object']).empty:
        return x.min()
    if pd.DataFrame(stats_number).select_dtypes(include=['number']).empty:
        return x.str.len().max()

assert get_stats(pd.Series(['a', 'bbbb', 'ccc'])) == 4
assert get_stats(pd.Series([1, 3, 2])) == 1

In [18]:
recipes.apply(get_stats) #раньше мы меняли submitted в формат времени, однако здесь я не поняла надо ли

name                83.0
id                  48.0
minutes              0.0
contributor_id    1530.0
submitted           10.0
n_steps              1.0
description       6321.0
n_ingredients        1.0
dtype: float64

In [19]:
recipes['submitted'] = pd.to_datetime(recipes['submitted'])
recipes.apply(get_stats) #поэтому тут приведен пример, когда submitted - это время
#я взяла, что тогда этот столбец будет nan, ибо мы не знаем, что с ним делать

name                83.0
id                  48.0
minutes              0.0
contributor_id    1530.0
submitted            NaN
n_steps              1.0
description       6321.0
n_ingredients        1.0
dtype: float64

### Группировки таблиц `pd.DataFrame`

5.1 Посчитайте количество рецептов, представленных каждым из участников (`contributor_id`). Какой участник добавил максимальное кол-во рецептов?

In [20]:
recipes.groupby(['contributor_id'])['contributor_id'].count()

contributor_id
1530            5
1533          186
1534           50
1535           40
1538            8
             ... 
2001968497      2
2002059754      1
2002234079      1
2002234259      1
2002247884      1
Name: contributor_id, Length: 8404, dtype: int64

In [21]:
recipes.groupby(['contributor_id'])['contributor_id'].count().idxmax() 
#участник 89831 добавил максимальное кол-во рецептов

89831

5.2 Посчитайте средний рейтинг к каждому из рецептов. Для скольких рецептов отсутствуют отзывы? Обратите внимание, что отзыв с нулевым рейтингом или не заполненным текстовым описанием не считается отсутствующим.

In [25]:
ratings = reviews.groupby(['recipe_id'])['rating'].mean()
ratings

recipe_id
48        1.000000
55        4.750000
66        4.944444
91        4.750000
94        5.000000
            ...   
536547    5.000000
536610    0.000000
536728    4.000000
536729    4.750000
536747    0.000000
Name: rating, Length: 28100, dtype: float64

In [26]:
len(recipes) - len(ratings)
#для 1900

1900

5.3 Посчитайте количество рецептов с разбивкой по годам создания.

In [27]:
recipes.groupby([recipes.submitted.dt.year])['contributor_id'].agg('count')

submitted
1999     275
2000     104
2001     589
2002    2644
2003    2334
2004    2153
2005    3130
2006    3473
2007    4429
2008    4029
2009    2963
2010    1538
2011     922
2012     659
2013     490
2014     139
2015      42
2016      24
2017      39
2018      24
Name: contributor_id, dtype: int64

5.4 Напишите функцию, которая принимает на вход таблицу (аналогичную `recipes` по набору столбцов), и возвращает `True` в том случае, если в столбце `minutes` присутствуют только значения, меньшие либо равные 10. Сгруппируйте таблицу `recipes` по полю `contributor_id` и для каждого участника выясните, справедливо ли, что все его рецепты занимают не более 10 минут.

In [29]:
def has_only_fast_recipes(x: pd.DataFrame) -> bool:
    return x.minutes.max() <= 10

assert not has_only_fast_recipes(
    pd.DataFrame(
        {
            "name": {0: "george s", 1: "healthy"},
            "id": {0: 44123, 1: 67664},
            "minutes": {0: 90, 1: 10},
            "contributor_id": {0: 35193, 1: 91970},
            "submitted": {0: "2002-10-25", 1: "2003-07-26"},
            "n_steps": {0: np.nan, 1: np.nan},
            "description": {0: "123", 1: "zxc"},
            "n_ingredients": {0: 18.0, 1: np.nan},
        }
    )
)
assert has_only_fast_recipes(
    pd.DataFrame(
        {
            "name": {0: "george s", 1: "healthy"},
            "id": {0: 44123, 1: 67664},
            "minutes": {0: 7, 1: 5},
            "contributor_id": {0: 35193, 1: 91970},
            "submitted": {0: "2002-10-25", 1: "2003-07-26"},
            "n_steps": {0: np.nan, 1: np.nan},
            "description": {0: "123", 1: "zxc"},
            "n_ingredients": {0: 18.0, 1: np.nan},
        }
    )
)

In [30]:
recipes.groupby(['contributor_id']).apply(has_only_fast_recipes)

contributor_id
1530          False
1533          False
1534          False
1535          False
1538          False
              ...  
2001968497    False
2002059754    False
2002234079    False
2002234259    False
2002247884    False
Length: 8404, dtype: bool

### Объединение таблиц `pd.DataFrame`

6.1 При помощи объединения таблиц, создайте `DataFrame`, состоящий из четырех столбцов: `id`, `name`, `user_id`, `rating`. Рецепты, на которые не оставлен ни один отзыв, должны отсутствовать в полученной таблице. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и попытавшись найти строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [31]:
recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


In [32]:
reviews.head()

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


In [37]:
recipes_merge = pd.merge(recipes, reviews, left_on = 'id', right_on = 'recipe_id', how = 'right')
recipes_merge = recipes_merge[['id', 'name', 'user_id', 'rating']]
recipes_merge

,id,name,user_id,rating
0,57993,salmon with tomatoes,21752,5
1,142201,lemon cajun stir fry,431813,5
2,252013,cottage cheese honey and cinnamon on toast,400708,4
3,404716,belly buttons,2001852463,5
4,129396,double tomato soup,95810,5
...,...,...,...,...
126691,335534,melissa s lemon pepper chicken,1270706,4
126692,8701,should be illegal oven bbq ribs,2282344,0
126693,222001,cheesy bacon bread,689540,5
126694,354979,kittencal s pan seared steak stove top to ove...,2000242659,5


In [56]:
recipes[recipes.id == 196634] 
#рецепт существует

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
8193,creamy cajun chicken with pasta,196634,45,377415,2006-11-19,NaN,quick pasta dish with a nice sauce,15.0


In [57]:
reviews[reviews.recipe_id == 196634]
#отзыва на него нет

,user_id,recipe_id,date,rating,review


In [55]:
recipes_merge[recipes_merge.id == 196634]
#в нашей таблице нет

,id,name,user_id,rating


6.2 При помощи объединения таблиц и группировок, создайте `DataFrame`, состоящий из трех столбцов: `recipe_id`, `name`, `review_count`, где столбец `review_count` содержит кол-во отзывов, оставленных на рецепт `recipe_id`. У рецептов, на которые не оставлен ни один отзыв, в столбце `review_count` должен быть указан 0. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и найдя строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [40]:
reviews_count = reviews.groupby('recipe_id')['recipe_id'].count()
reviews_count

recipe_id
48         2
55         4
66        18
91         4
94         4
          ..
536547     1
536610     1
536728     1
536729     4
536747     4
Name: recipe_id, Length: 28100, dtype: int64

In [41]:
reviews_merge = pd.merge(recipes, reviews_count, how = 'left', left_on = 'id', right_index = True)
reviews_merge = reviews_merge[['id', 'name', 'recipe_id']]
reviews_merge.columns = ['recipe_id', 'name', 'review_count']
reviews_merge['review_count'] = reviews_merge['review_count'].fillna(0)
reviews_merge

,recipe_id,name,review_count
0,44123,george s at the cove black bean soup,3.0
1,67664,healthy for them yogurt popsicles,8.0
2,38798,i can t believe it s spinach,3.0
3,35173,italian gut busters,1.0
4,84797,love is in the air beef fondue sauces,8.0
...,...,...,...
29995,267661,zurie s holey rustic olive and cheddar bread,4.0
29996,386977,zwetschgenkuchen bavarian plum cake,2.0
29997,103312,zwiebelkuchen southwest german onion cake,6.0
29998,486161,zydeco soup,6.0


In [58]:
recipes[recipes.id == 196634]
#рецепт существует

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
8193,creamy cajun chicken with pasta,196634,45,377415,2006-11-19,NaN,quick pasta dish with a nice sauce,15.0


In [59]:
reviews[reviews["recipe_id"] == 196634]
#отзывов нет

,user_id,recipe_id,date,rating,review


In [60]:
reviews_merge[reviews_merge["recipe_id"] == 196634]
#здесь есть

,recipe_id,name,review_count
8193,196634,creamy cajun chicken with pasta,0.0


6.3. Выясните, рецепты, добавленные в каком году, имеют наименьший средний рейтинг?

In [45]:
recipes_merge_full = pd.merge(recipes, reviews, left_on = 'id', right_on = 'recipe_id', how = 'right')
recipes_rating = recipes_merge_full.groupby(recipes_merge_full.submitted.dt.year)["rating"].agg('mean')
recipes_rating

submitted
1999    4.274895
2000    4.284585
2001    4.393945
2002    4.404645
2003    4.439152
2004    4.463778
2005    4.409687
2006    4.416310
2007    4.420073
2008    4.387416
2009    4.360447
2010    4.406279
2011    4.375850
2012    4.412736
2013    4.336508
2014    4.538235
2015    4.207317
2016    3.538462
2017    2.750000
2018    3.388889
Name: rating, dtype: float64

In [46]:
recipes_rating.idxmin()
#в 2017

2017

### Сохранение таблиц `pd.DataFrame`

7.1 Отсортируйте результат выполнения задания 6.1 в порядке убывания величины столбца `id` и сохраните результаты в csv файл. 

In [47]:
recipes_merge.sort_values("id", ascending=False).to_csv("6.1_results.csv", encoding="utf8")

7.2 Воспользовавшись `pd.ExcelWriter`, cохраните результаты 6.1 и 6.2 в файл: на лист с названием `Рецепты с оценками` сохраните результаты выполнения 6.1; на лист с названием `Количество отзывов по рецептам` сохраните результаты выполнения 6.2.

In [51]:
with pd.ExcelWriter("6.1_an_6.2_results.xlsx") as writer:
    recipes_merge.to_excel(writer, sheet_name="Рецепты с оценками", encoding="utf8")
    reviews_merge.to_excel(writer, sheet_name="Количество отзывов по рецептам", encoding="utf8")

/opt/anaconda3/envs/test/lib/python3.8/site-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [50]:
!pip install openpyxl

  Using cached openpyxl-3.0.10-py2.py3-none-any.whl (242 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
